In [1]:
# Neural networks can be constructed using the torch.nn package.

# Define the network
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        print(x.size())
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        print(x.size())
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print(x.size())
        x = x.view(-1, self.num_flat_features(x))
        print(x.size())
        x = F.relu(self.fc1(x))
        print(x.size())
        x = F.relu(self.fc2(x))
        print(x.size())
        x = self.fc3(x)
        print(x.size())
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)


params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
# print('----------------')
# for each in params:
#   print(each.size())
# print('----------------')
# print(params[0])
# print('**************')
# print(params[-1])


#Let’s try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

input = torch.randn(1,1,32,32)
print(input)
out = net(input)
print(out)

# Zero the gradient buffers of all parameters and backprops with random gradients:
net.zero_grad()
out.backward(torch.randn(1, 10))


### Loss Function ###
# A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1,-1)  # make it the same shape as output
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### Backprop ###
# To backpropagate the error all we have to do is to loss.backward(). 
# You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.
# Now we shall call loss.backward(), and have a look at conv1’s bias gradients before and after the backward.
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv2.bias) ##### temp
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

# Update the weights
# (weight = weight - learning_rate * gradient)
print(net.parameters)
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

# However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
# To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

import torch.optim as optim
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
print(optimizer)
# in your training loop:
optimizer.zero_grad()  # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

#print(net.state_dict())

# NOTE: Observe how gradient buffers had to be manually set to zero using optimizer.zero_grad().
# This is because gradients are accumulated as explained in the Backprop section.

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
torch.Size([6, 1, 3, 3])
tensor([[[[-0.8496,  0.1757,  0.1044,  ..., -0.9030,  0.3680,  0.6075],
          [-0.9557,  0.7580,  1.1072,  ...,  1.1732,  0.4037, -0.7146],
          [-1.3369,  0.2783, -2.2107,  ...,  0.8581,  1.4845,  0.6992],
          ...,
          [ 0.1136,  0.8096, -1.1722,  ...,  0.1692, -0.5044,  0.0439],
          [ 0.5649, -0.2376, -0.5716,  ..., -0.5036,  1.0726,  1.0586],
          [-0.0507, -0.3487,  1.1868,  ..., -1.0833, -1.4617, -0.4981]]]])
torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 15, 15])
torch.Size([1, 16, 6, 6])
torch.Size([1, 576])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])
tensor([[-0.0086, -0.0022, -0.0154, -0.0584,  0.007